In [1]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 34.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
google-cola

In [2]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch
from torchvision import transforms
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from roboflow import Roboflow
import roboflow.core.workspace  # For workspace-related methods
import requests

In [3]:
# Step 1: Define paths
base_dir = "/kaggle/input/pestopia/Datasets/Pest_Dataset"
pesticide_csv = "/kaggle/input/pestopia/Datasets/Pesticide_Dataset/Pesticides.csv"

In [4]:
# Step 2: Analyze the dataset structure and check for missing files
def analyze_dataset(base_dir):
    pest_classes = os.listdir(base_dir)  # List of pest class directories (e.g., Adristyrannus)
    pest_classes = [p for p in pest_classes if os.path.isdir(os.path.join(base_dir, p))]
    print(f"Total number of pest classes: {len(pest_classes)}")

    # Dictionary to store image paths and labels
    image_paths = []
    labels = []
    label_to_idx = {pest: idx for idx, pest in enumerate(pest_classes)}
    missing_files = 0
    corrupted_files = 0

    for pest in tqdm(pest_classes, desc="Analyzing dataset"):
        pest_dir = os.path.join(base_dir, pest)
        if not os.path.isdir(pest_dir):
            print(f"Directory missing: {pest_dir}")
            continue

        for img_name in os.listdir(pest_dir):
            img_path = os.path.join(pest_dir, img_name)
            try:
                # Try to open the image to check if it's valid
                with Image.open(img_path) as img:
                    img.verify()  # Verify image integrity
                    img = Image.open(img_path).convert('RGB')  # Reopen to ensure it can be loaded
                image_paths.append(img_path)
                labels.append(label_to_idx[pest])
            except Exception as e:
                print(f"Corrupted image: {img_path}, Error: {e}")
                corrupted_files += 1
                missing_files += 1

    print(f"Total images found: {len(image_paths)}")
    print(f"Missing or corrupted files: {missing_files}")
    print(f"Corrupted files: {corrupted_files}")
    return image_paths, labels, label_to_idx

# Run the analysis
image_paths, labels, label_to_idx = analyze_dataset(base_dir)

Total number of pest classes: 132


Analyzing dataset:  92%|█████████▏| 122/132 [05:26<00:23,  2.31s/it]

Corrupted image: /kaggle/input/pestopia/Datasets/Pest_Dataset/bollworm/bollworm, Error: [Errno 21] Is a directory: '/kaggle/input/pestopia/Datasets/Pest_Dataset/bollworm/bollworm'


Analyzing dataset: 100%|██████████| 132/132 [05:46<00:00,  2.63s/it]

Total images found: 55864
Missing or corrupted files: 1
Corrupted files: 1


In [5]:
# Step 3: Load pesticide CSV
pesticide_df = pd.read_csv(pesticide_csv)
print("\nPesticide CSV Preview:")
print(pesticide_df.head())
print(f"Unique pests in CSV: {pesticide_df['Pest Name'].nunique()}")


Pesticide CSV Preview:
                  Pest Name                   Most Commonly Used Pesticides
0             Adristyrannus                 Acephate, Malathion, Permethrin
1  Aleurocanthus spiniferus         Imidacloprid, Acetamiprid, Chlorpyrifos
2         alfalfa plant bug  Imidacloprid, Lambda-cyhalothrin, Chlorpyrifos
3      alfalfa seed chalcid            Azadirachtin, Imidacloprid, Spinosad
4            alfalfa weevil              Chlorpyrifos, Cyfluthrin, Carbaryl
Unique pests in CSV: 132


In [6]:
base_dir = "/kaggle/input/pestopia/Datasets/Pest_Dataset"
subset_dir = "/kaggle/working/pestopia_subset"
flat_subset_dir = "/kaggle/working/pestopia_subset_flat"
os.makedirs(subset_dir, exist_ok=True)
os.makedirs(flat_subset_dir, exist_ok=True)

In [7]:
def collect_image_paths(base_dir):
    pest_classes = os.listdir(base_dir)
    pest_classes = [p for p in pest_classes if os.path.isdir(os.path.join(base_dir, p))]
    image_paths = []
    labels = []

    for pest in tqdm(pest_classes, desc="Collecting image paths"):
        pest_dir = os.path.join(base_dir, pest)
        for img_name in os.listdir(pest_dir):
            img_path = os.path.join(pest_dir, img_name)
            image_paths.append(img_path)
            labels.append(pest)

    return image_paths, labels, pest_classes

image_paths, labels, pest_classes = collect_image_paths(base_dir)
print(f"Total images: {len(image_paths)}")
print(f"Total pest classes: {len(pest_classes)}")

Total images: 55865
Total pest classes: 132


In [8]:
target_images = 10000  # Reduced subset for faster annotation
proportion = target_images / len(image_paths)
images_per_class = {pest: 0 for pest in pest_classes}

# Count images per class
for label in labels:
    images_per_class[label] += 1

# Target images per class (minimum 1 image per class)
target_per_class = {}
for pest in pest_classes:
    current_count = images_per_class[pest]
    target_count = max(1, int(current_count * proportion))
    target_per_class[pest] = target_count

# Adjust to ensure total is exactly 1,000
total_target = sum(target_per_class.values())
if total_target != target_images:
    scaling_factor = target_images / total_target
    for pest in target_per_class:
        target_per_class[pest] = max(1, int(target_per_class[pest] * scaling_factor))

total_target = sum(target_per_class.values())
if total_target > target_images:
    excess = total_target - target_images
    sorted_classes = sorted(target_per_class.items(), key=lambda x: x[1], reverse=True)
    for pest, count in sorted_classes:
        if excess <= 0:
            break
        reduce_by = min(excess, count - 1)
        target_per_class[pest] -= reduce_by
        excess -= reduce_by

# Step 4: Sample images for the subset
subset_paths = []
subset_labels = []

for pest in tqdm(pest_classes, desc="Creating subset"):
    pest_paths = [p for p, l in zip(image_paths, labels) if l == pest]
    target_count = target_per_class[pest]
    if len(pest_paths) <= target_count:
        selected_paths = pest_paths
    else:
        selected_paths = pest_paths[:target_count]
    subset_paths.extend(selected_paths)
    subset_labels.extend([pest] * len(selected_paths))

print(f"Subset created: {len(subset_paths)} images")

# Step 5: Copy images to subset directory with class subdirectories
for img_path, label in tqdm(zip(subset_paths, subset_labels), total=len(subset_paths), desc="Copying images to subset"):
    class_dir = os.path.join(subset_dir, label)
    os.makedirs(class_dir, exist_ok=True)
    img_name = os.path.basename(img_path)
    dest_path = os.path.join(class_dir, img_name)
    shutil.copy(img_path, dest_path)

print(f"Subset copied to {subset_dir}")

Creating subset: 100%|██████████| 132/132 [00:00<00:00, 567.36it/s]


Subset created: 9952 images


Copying images to subset: 100%|██████████| 9952/9952 [00:11<00:00, 849.14it/s]

Subset copied to /kaggle/working/pestopia_subset


In [9]:
# Step 6: Flatten the directory structure for Roboflow upload
for label in tqdm(subset_labels, desc="Flattening directory"):
    class_dir = os.path.join(subset_dir, label)
    for img_name in os.listdir(class_dir):
        src_path = os.path.join(class_dir, img_name)
        # Rename the image to include the class name (e.g., grasshopper_image1.jpg)
        base_name, ext = os.path.splitext(img_name)
        new_img_name = f"{label}_{base_name}{ext}"
        dest_path = os.path.join(flat_subset_dir, new_img_name)
        shutil.copy(src_path, dest_path)

flat_images = os.listdir(flat_subset_dir)
print(f"Flattened subset created: {len(flat_images)} images in {flat_subset_dir}")

Flattening directory: 100%|██████████| 9952/9952 [06:00<00:00, 27.60it/s]  

Flattened subset created: 9952 images in /kaggle/working/pestopia_subset_flat


In [12]:
# Step 1: Define paths
flat_subset_dir = "/kaggle/working/pestopia_subset_flat"
zip_file_path = "/kaggle/working/pestopia_subset_flat.zip"

# Step 2: Verify the contents of the flattened subset directory
if not os.path.exists(flat_subset_dir):
    raise FileNotFoundError(f"Directory {flat_subset_dir} does not exist. Ensure you have run the previous steps to create the flattened subset.")

flat_images = [f for f in os.listdir(flat_subset_dir) if os.path.isfile(os.path.join(flat_subset_dir, f))]
print(f"Found {len(flat_images)} images in {flat_subset_dir}")

if len(flat_images) == 0:
    raise ValueError(f"No images found in {flat_subset_dir}. Ensure the directory contains image files.")

# Step 3: Create a ZIP file of the flattened subset
print(f"Creating ZIP file at {zip_file_path}...")
shutil.make_archive(
    base_name="/kaggle/working/pestopia_subset_flat",  # Base name for the ZIP file (without .zip extension)
    format="zip",  # Format of the archive
    root_dir=flat_subset_dir  # Directory to zip
)

# Step 4: Verify the ZIP file was created
if not os.path.exists(zip_file_path):
    raise FileNotFoundError(f"Failed to create ZIP file at {zip_file_path}.")

zip_size = os.path.getsize(zip_file_path) / (1024 * 1024)  # Size in MB
print(f"ZIP file created: {zip_file_path} ({zip_size:.2f} MB)")

Found 9952 images in /kaggle/working/pestopia_subset_flat
Creating ZIP file at /kaggle/working/pestopia_subset_flat.zip...
ZIP file created: /kaggle/working/pestopia_subset_flat.zip (393.15 MB)


In [10]:
# Step 7: Upload to Roboflow with verification and error handling
api_key = "LGaRRqVtNzMpwOIEe6oD"
workspace = "sus-ltlqs"  # This might be incorrect; we'll verify
project_name = "pestopia-annotation"

try:
    # Initialize Roboflow
    rf = Roboflow(api_key=api_key)
    
    # Step 7.1: Verify the workspace
    print("Verifying workspace...")
    workspaces = rf.workspaces()
    workspace_names = [w["name"] for w in workspaces]
    print(f"Available workspaces: {workspace_names}")
    
    if workspace not in workspace_names:
        raise ValueError(f"Workspace '{workspace}' not found. Available workspaces: {workspace_names}. Please check your workspace name or API key permissions.")
    
    # Step 7.2: Check if the project exists, create if it doesn't
    print(f"Checking for project '{project_name}' in workspace '{workspace}'...")
    workspace_obj = rf.workspace(workspace)
    projects = workspace_obj.projects()
    project_names = [p["name"] for p in projects]
    print(f"Existing projects in workspace '{workspace}': {project_names}")
    
    if project_name not in project_names:
        print(f"Project '{project_name}' does not exist. Creating it...")
        project = workspace_obj.create_project(
            project_name=project_name,
            project_type="object-detection",  # For bounding box annotations
            annotation="pest-classification"
        )
        print(f"Created project: {project_name}")
    else:
        project = workspace_obj.project(project_name)
        print(f"Project '{project_name}' already exists. Using existing project.")

    # Step 7.3: Upload the flattened dataset
    print(f"Uploading {len(flat_images)} images to Roboflow project: {project_name}")
    project.upload(flat_subset_dir, num_workers=10)  # Upload with 10 concurrent workers
    print(f"Successfully uploaded {len(flat_images)} images to Roboflow project: {project_name}")

except Exception as e:
    print(f"Error during Roboflow upload: {e}")
    print("Troubleshooting steps:")
    print("1. Verify your API key is correct and has permissions.")
    print("2. Check if the workspace name is correct (run `rf.workspaces()` to list available workspaces).")
    print("3. Ensure the project exists or can be created.")
    print("4. Verify that {flat_subset_dir} contains only image files (no subdirectories).")
    print("5. Check your internet connection and Roboflow's status.")

Verifying workspace...
Error during Roboflow upload: 'Roboflow' object has no attribute 'workspaces'
Troubleshooting steps:
1. Verify your API key is correct and has permissions.
2. Check if the workspace name is correct (run `rf.workspaces()` to list available workspaces).
3. Ensure the project exists or can be created.
4. Verify that {flat_subset_dir} contains only image files (no subdirectories).
5. Check your internet connection and Roboflow's status.
